# Predicting daily bikerider count from temperature and weekday

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import statsmodels.api as sm
import statsmodels.formula.api as smf

import pymc3 as pm
import arviz as az

import scipy.stats as stats

# import custom functions for data loading, plotting, helpers for model building etc.
import sys  
sys.path.insert(0, './../modules')
import m_data_loading as mdl
import m_plotting as mplot
import m_helpers
import m_check

## Import data

We import a dataset that contains the daily number of bike riders at a counting station in Freiburg (channel name: 'FR1 Dreisam / Hindenburgstr.') in 2021 as well as the daily average temperature and a boolean whether a given day is a business day.

In [ ]:
# counting station and time period
city_name = 'Stadt Freiburg'
counter_site = 'FR1 Dreisam / Otto-Wels-Str.'
channel_name = 'FR1 Dreisam / Hindenburgstr.'
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 12, 31)

# import data
combined_daily_dat = mdl.get_data_for_location(city_name, counter_site, channel_name, start_date, end_date)

In [ ]:
# inspect
combined_daily_dat

In [ ]:
# plot daily bikerider count by mean day temperature, separate between business days and weekends
plt.figure(figsize=(9,7))
plt.title(city_name + '\ncounter site: ' + counter_site + '\nchannel name: ' + channel_name);
mplot.plot_data()

## Statistical Analysis: Multiple Linear Regression

Independent variables:  
$x_1$: daily average temperature [°C]  
$x_2$: indicator variable whether day is a business day $\{0,1\}$

Dependent variable:  
$y$: daily bike rider count


Multiple linear regression:  

$$\mu = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2$$
$$y \sim \mathcal{N}(\mu, \sigma) $$


$\beta_0$: intercept on non-business days (i.e., bike rider count at $0°C$ on the weekend)  
$\beta_1$: slope on non-business days (weekends)  
$\beta_2$: additional intercept on business days  
$\beta_3$: interaction parameter (additional slope on business days)

So:
- on weeekends (i.e., $x_2 = 0$): $$\mu = \beta_0 + \beta_1 x_1$$
- on business days (i.e., $x_2 = 1$): $$\mu = (\beta_0 + \beta_2) + (\beta_1 + \beta_3) x_1$$

### Gaussian data distribution

In [ ]:
def build_linear_model_with_normal_data_distrib(prior_config, dat=combined_daily_dat):
    '''
    Build a linear model with categories "business-day vs no-business-day",
    using the given parameter values for the prior distributions and a normal
    distribution as data distribution.

    Parameters
    ----------
    prior_config : dict
        Dictionary that contains the parameter values of the prior distributions.
    dat : pd.DataFrame
        Dataframe with columns 'temperature', 'is_busday' and 'rider_count'.
        Default: combined_daily_dat

    Returns
    -------
    model : pm.Model
        Model
    idata : arviz.InferenceData
        samples from the posterior
    '''
    # build linear model with categories "business-day vs no-business-day"
    with pm.Model() as model:

        # data
        dat_temps = pm.Data('dat_temps', dat[['temperature']])
        dat_busday = pm.Data('dat_busday', dat[['is_busday']])
        dat_rider_counts = pm.Data('dat_rider_counts', dat[['rider_count']])
        
        # prior for parameters beta
        # idea: same parametrization as in Frequentist analysis above.
        if (prior_config['truncate_intercept_prior']): # truncate intercept prior: >= 0.
            beta_intercept = pm.TruncatedNormal("beta_intercept", mu=prior_config['mu_intercept_prior'], sigma=prior_config['sigma_intercept_prior'], lower=0, shape=1)
        else:
            beta_intercept = pm.Normal("beta_intercept", mu=prior_config['mu_intercept_prior'], sigma=prior_config['sigma_intercept_prior'], shape=1)
        beta_slope = pm.Normal("beta_slope", mu=prior_config['mu_slope_prior'], sigma=prior_config['sigma_slope_prior'], shape=1)
        beta_intercept_business_days = pm.Normal("beta_intercept_business_days", mu=prior_config['mu_intercept_business_days_prior'], sigma=prior_config['sigma_intercept_business_days_prior'], shape=1)
        beta_slope_interaction = pm.Normal("beta_slope_interaction", mu=prior_config['mu_slope_interaction_prior'], sigma=prior_config['sigma_slope_interaction_prior'], shape=1)

        # prior for two noise distributions: better small
        sigma = pm.Exponential("sigma", lam=.01, shape=2)
            
        # the linear model function
        expected = beta_intercept + beta_slope * dat_temps + beta_intercept_business_days * dat_busday + beta_slope_interaction * dat_temps * dat_busday
            
        # Data and Likelihood: Gaussian noise
        noise = sigma[0] * (1 - dat_busday) + sigma[1] * dat_busday
        # TODO use pm.TruncatedNormal with lower=0; but makes prediction slow.
        obs = pm.Normal("observed",
                        mu=expected,
                        sigma=noise,
                        observed=dat_rider_counts)

        # sample from posterior
        idata = pm.sample(5000, chains=4, tune=3000, target_accept=0.9, return_inferencedata=True)

    return model, idata

We know:
- daily bike count $\geq 0$ (for "usual" temperature range, i.e., -10°C to 30°C, the model should reflect this)
- daily bike rider count increases with temperature
- people rather cancel/reschedule a recreational bike ride due to weather than their daily commute (known from literature review and personal experience) (TODO nochmal diskutieren, ob wir das als Vorwissen definieren)
- Population of Freiburg $\approx 230.000$.

--> Define prior distributions for the model parameters that reflect this prior knowledge.

TODO Kommentar: Build model...

In [ ]:
# build linear model with categories "business-day vs no-business-day"
linearCategoricalModel_prior_config = m_helpers.create_prior_config_dict(4000, 2000, 300, 300, 0, 2000, -30, 100, True)
linearCategoricalModel, linearCategoricalModel_idata, = build_linear_model_with_normal_data_distrib(linearCategoricalModel_prior_config)

Prior predictive check: plot what the priors mean / which regression lines are favored/allowed by priors:

In [ ]:
with linearCategoricalModel:
    # generate samples from the prior predictive distribution
    linearCategoricalModel_prior_checks = pm.sample_prior_predictive(samples=50, random_seed=123)

mplot.prior_pred_check(linearCategoricalModel_prior_checks)

Show summary, traces, posterior distributions, as well as regression lines sampled from posterior:

In [ ]:
mplot.show_results_linear_fit(linearCategoricalModel_idata)

In [ ]:
# compute the posterior distribution for beta_slope + beta_slope_interaction,
# i.e., the slope parameter on business days
beta_slope_post_chains = linearCategoricalModel_idata.posterior['beta_slope'].values
beta_slope_interaction_post_chains = linearCategoricalModel_idata.posterior['beta_slope_interaction'].values
beta_slope_weekend_chains = beta_slope_post_chains + beta_slope_interaction_post_chains

# plot the posterior distribution
az.plot_posterior(beta_slope_weekend_chains, hdi_prob=0.95);

# display summary
az.summary(beta_slope_weekend_chains)

#### Prior sensitivity analysis

In [ ]:
# TODO Kommentar woanders nutzen
# build linear model with categories "business-day vs no-business-day", using
# prior distributions for intercept and slope with positive mu, and a prior
# distribution for the interaction that has a negative mu.
# We keep the prior distribution for the additional intercept for business days 
# zero-centered because we don't know enough about Freiburg to decide whether
# the counter site as a "weekend-trip-location" or a "commuting-location".
# ---
# TODO Maybe try model with bi-modal prior for the additional intercept on
# business days because: we know (from personal experience and literature) that
# there is a difference between weekend and business day - but (due to lack of
# local knowledge about Freiburg) we don't know in which direction.
# ---

##### Alternative model 1: Zero-centered priors

Main difference to original model: Use zero-centered priors. Additionally, priors are wider than in original model.

In [ ]:
# build linear model with categories "business-day vs no-business-day", using
# zero-centered prior distributions
alt_prior_config_1 = m_helpers.create_prior_config_dict(0, 4000, 0, 500, 0, 2000, 0, 150)
linearCategoricalModel_alt_prior_config_1, linearCategoricalModel_alt_prior_config_1_idata = build_linear_model_with_normal_data_distrib(alt_prior_config_1)

# generate samples from the prior predictive distribution
with linearCategoricalModel_alt_prior_config_1:
    linearCategoricalModel_alt_prior_config_1_prior_checks = pm.sample_prior_predictive(samples=50, random_seed=123)

##### Alternative model 2: Zero-centered priors

Difference to alternative model 1: Prior distributions are more narrow.

In [ ]:
# build linear model with categories "business-day vs no-business-day", using
# zero-centered prior distributions
alt_prior_config_2 = m_helpers.create_prior_config_dict(0, 1000, 0, 100, 0, 1000, 0, 100)
linearCategoricalModel_alt_prior_config_2, linearCategoricalModel_alt_prior_config_2_idata = build_linear_model_with_normal_data_distrib(alt_prior_config_2)

# generate samples from the prior predictive distribution
with linearCategoricalModel_alt_prior_config_2:
    linearCategoricalModel_alt_prior_config_2_prior_checks = pm.sample_prior_predictive(samples=50, random_seed=123)

##### Alternative model 3: Different slope prior

Difference to original model: prior for slope parameter is less positive and more narrow.

In [ ]:
# build linear model with categories "business-day vs no-business-day"
alt_prior_config_3 = m_helpers.create_prior_config_dict(4000, 2000, 100, 100, 0, 2000, -30, 100, True)
linearCategoricalModel_alt_prior_config_3, linearCategoricalModel_alt_prior_config_3_idata = build_linear_model_with_normal_data_distrib(alt_prior_config_3)

# generate samples from the prior predictive distribution
with linearCategoricalModel_alt_prior_config_3:
    linearCategoricalModel_alt_prior_config_3_prior_checks = pm.sample_prior_predictive(samples=50, random_seed=123)

##### Compare posterior distributions between the models that were built above (using different parameter values for the prior).

Summarize data of the different model instances built above in one list.

In [ ]:
# summarize idata, prior configuration and prior predictive checks of all models
# in one list
models_dat = [
    {'name': 'alternative model 1', 'idata': linearCategoricalModel_alt_prior_config_1_idata, 'prior_config': alt_prior_config_1, 'prior_checks': linearCategoricalModel_alt_prior_config_1_prior_checks},
    {'name': 'alternative model 2', 'idata': linearCategoricalModel_alt_prior_config_2_idata, 'prior_config': alt_prior_config_2, 'prior_checks': linearCategoricalModel_alt_prior_config_2_prior_checks},
    {'name': 'alternative model 3', 'idata': linearCategoricalModel_alt_prior_config_3_idata, 'prior_config': alt_prior_config_3, 'prior_checks': linearCategoricalModel_alt_prior_config_3_prior_checks},
    {'name': 'model', 'idata': linearCategoricalModel_idata, 'prior_config': linearCategoricalModel_prior_config, 'prior_checks': linearCategoricalModel_prior_checks}
]

Compare prior predictive checks: Plot what the priors mean / which regression lines are favored/allowed by priors.

In [ ]:
# indices of the models for which to plot the prior predictive checks
model_indices = [0,1,2,3]

# initialize figure
plt.figure(figsize=(20, 5))

for idx, model_idx in enumerate(model_indices):
    plt.subplot(1, len(model_indices), idx+1)
    mplot.prior_pred_check(models_dat[model_idx]['prior_checks'], title='Prior predictive checks for ' + models_dat[model_idx]['name'])

plt.tight_layout()

Compare MAP and example credible regression lines.

In [ ]:
# indices of the models for which to plot the MAP and example posteriors
model_indices = [0,1,2,3]

# initialize figure
plt.figure(figsize=(20, 5))

for idx, model_idx in enumerate(model_indices):
    plt.subplot(1, len(model_indices), idx+1)
    mplot.show_data_MAP_and_posts(models_dat[model_idx]['idata'], mplot.plot_linear_fit)
    plt.title('MAP and example posteriors for ' + models_dat[model_idx]['name'])

plt.tight_layout()

Compare posterior distributions.

In [ ]:
# plot posterior distributions
# compare posterior distributions between the models that were built above
# (using different parameter values for the prior)
mplot.plot_posteriors(models_dat, [0, 1, 2, 3], var_names=['beta_intercept', 'beta_slope', 'beta_intercept_business_days', 'beta_slope_interaction'], plot_hdi=True)

--> All of the prior parameter values tested above yield roughly the same posterior distributions for the parameters of the regression lines.

Compare prior and posterior distributions for all models.

In [ ]:
for var_name in ['beta_intercept', 'beta_slope', 'beta_intercept_business_days', 'beta_slope_interaction']:
    mplot.compare_models_priors_and_posteriors(models_dat, [0, 1, 2, 3], var_name)

#### Posterior predictive check

TODO check model assumptions: independence, linear relationship, equal variance/normal distribution, ...

In [ ]:
linCatMod_ppc, linCatMod_posterior_predicitive_means, linCatMod_errors, linCatMod_median_absolute_error, linCatMod_scaled_MAE, linCatMod_quants_per_datapoint_df, linCatMod_percent_within_50, linCatMod_percent_within_95 = m_check.post_pred_check(linearCategoricalModel, linearCategoricalModel_idata, print_output=True, plot_pred_ints=True)

In [ ]:
# plot prediction intervals together with original datapoints
# TODO später entfernen

# extract predicted bike rider counts (for business days and non-business days)
preds = linCatMod_ppc['observed'][:,:,0]
preds_busdays = preds[:, combined_daily_dat[combined_daily_dat['is_busday']].index]
preds_nonbusdays = preds[:, combined_daily_dat[~combined_daily_dat['is_busday']].index]

# extract temperatures of datapoints (for business days and non-business days)
temps_busdays = combined_daily_dat[combined_daily_dat['is_busday']]['temperature']
temps_nonbusdays = combined_daily_dat[~combined_daily_dat['is_busday']]['temperature']

print('preds shape: ', np.shape(preds))
print('busday preds shape: ', np.shape(preds_busdays))
print('nonbusday preds shape: ', np.shape(preds_nonbusdays))
print('busday temps shape: ', np.shape(temps_busdays))
print('nonbusday temps shape: ', np.shape(temps_nonbusdays))

# plot
az.plot_hdi(temps_busdays, preds_busdays, color='orange') # TODO label: 'HDI...'
az.plot_hdi(temps_nonbusdays, preds_nonbusdays, color='blue')
mplot.plot_data();

TODO

Im Großen und ganzen passt das, oder?
Zum Beispiel:
- variability in Originaldaten ungefähr wie variability in Vorhersagen.
- General positive trend in original and predicted data

Dass etwa 95% der Original-Datenpunkte im 95% Vorhersageintervall liegen, spricht dafür, dass das Modell passt, oder?

Wobei etwa um 5-10°C "Ausschlag nach unten" nicht vorhergesagt wird...

In [ ]:
# compare to MAE if we would always predict overall mean:
print('Median absolute error when predicting mean: ', np.median(np.abs(combined_daily_dat['rider_count'] - np.mean(combined_daily_dat['rider_count']))))
print('standard deviations of absolute errors when predicting mean:', np.std(np.abs(combined_daily_dat['rider_count'] - np.mean(combined_daily_dat['rider_count']))))
print('standard deviations of errors:', np.std(combined_daily_dat['rider_count'] - np.mean(combined_daily_dat['rider_count'])))

TODO sind densities (siehe unten) anschaulich oder lieber regression lines mit prediction intervals (siehe oben)?

In [ ]:
# compare simulated and original data
def compare_sim_to_original(sims, original, xlabel):
    '''
    Parameters
    ----------
    sims : TODO
        simulated bike rider counts
    original : TODO
        original (observed) bike rider counts
    xlabel : str
        x-axis label

    Returns
    -------
    (plot)
    '''
    # initialize figure
    plt.figure()

    # simulated bike rider counts data
    for i in np.arange(50):
        if i==0: # with label
            az.plot_kde(sims[i,:], label='simulated', plot_kwargs={'color': 'grey'})
        else:
            az.plot_kde(sims[i,:], plot_kwargs={'color': 'grey'})

    # original bike rider counts data
    az.plot_kde(original, label='original', plot_kwargs={'color': 'red'})

    # add axis label and legend
    plt.xlabel('Daily number of bike riders')
    plt.legend();

compare_sim_to_original(preds, combined_daily_dat['rider_count'], 'Daily number of bike riders')

# seperated for business days and non-business days
compare_sim_to_original(preds_busdays, combined_daily_dat[combined_daily_dat['is_busday']]['rider_count'], 'Daily number of bike riders on business days')
compare_sim_to_original(preds_nonbusdays, combined_daily_dat[~combined_daily_dat['is_busday']]['rider_count'], 'Daily number of bike riders on non-business days')

#### Cross validation

Check the model's predictive accuracy.

TODO random seeds setzen? Für pm.sample_posterior_predictive...

In [ ]:
# TODO adjust prior config
linearCategoricalModel_CV_MAEs, linearCategoricalModel_CV_scaledMAEs, linearCategoricalModel_CV_percent_within_50, linearCategoricalModel_CV_percent_within_95 = m_check.cross_validation(build_linear_model_with_normal_data_distrib, linearCategoricalModel_prior_config, num_folds=5, dat=combined_daily_dat, print_results=True)

TODO 95% coverage of 95% prediction interval ist gut, oder? Betrachte neue Daten....

TODO Vergleiche mit weiter oben, wo MAE, prediction interval coverage, ... für den ganzen Datensatz, auf dem trainiert wurde, berechnet wurden.

TODO run the same cross validation with poisson. Compare to model with normal distribution (above).
Or maybe use az.loo oder arviz.compare?

#### Test for new data

##### Test for same counting station, next year

In [ ]:
# specify counting station and time period
city_name = 'Stadt Freiburg'
counter_site = 'FR1 Dreisam / Otto-Wels-Str.'
channel_name = 'FR1 Dreisam / Hindenburgstr.'
test_start_date = datetime.date(2022, 1, 1)
test_end_date = datetime.date(2022, 12, 31)

# import data for counting station
test_data_next_year = mdl.get_data_for_location(city_name, counter_site, channel_name, test_start_date, test_end_date)

In [ ]:
# check data import
test_data_next_year

In [ ]:
# test model on data of next year
# compare the posterior predictive means with observed (test) data points
linCatMod_ppc_next_year, linCatMod_posterior_predicitive_means_next_year, linCatMod_errors_next_year, linCatMod_median_absolute_error_next_year, linCatMod_scaled_MAE_next_year, linCatMod_quants_per_datapoint_df_next_year, linCatMod_percent_within_50_next_year, linCatMod_percent_within_95_next_year = m_check.post_pred_check(linearCategoricalModel, linearCategoricalModel_idata, given_data = test_data_next_year, change_dat=True, print_output=True, plot_error_hist=True, plot_pred_ints=True, plot_post_pred_means=True)

TODO for business days: predicted values too small

TODO possible reason: Covid + city with university. In 2022, students are back in town.

##### Test for another counter site in same city

In [ ]:
# specify counting station and time period
test_city_name = 'Stadt Freiburg'
test_counter_site = 'FR2 Güterbahn / Ferd.-Weiß-Str.'
test_channel_name = 'FR2 Güterbahn / Ferd.-Weiß-Str.'
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 12, 31)
#test_channel_ID = 101014511
# TODO use channel_id for identification --> to be able to do so: export also channel_id when building the dataset

# extract data for counting station
test_data_new_counter_site = mdl.get_data_for_location(test_city_name, test_counter_site, test_channel_name, start_date, end_date)

In [ ]:
# check data import
test_data_new_counter_site

In [ ]:
# test model on data of another (new) counter site
# compare the posterior predictive means with observed (test) data points
linCatMod_ppc_new_counter_site, linCatMod_posterior_predicitive_means_new_counter_site, linCatMod_errors_new_counter_site, linCatMod_median_absolute_error_new_counter_site, linCatMod_scaled_MAE_new_counter_site, linCatMod_quants_per_datapoint_df_new_counter_site, linCatMod_percent_within_50_new_counter_site, linCatMod_percent_within_95_new_counter_site = m_check.post_pred_check(linearCategoricalModel, linearCategoricalModel_idata, given_data = test_data_new_counter_site, change_dat=True, print_output=True, plot_error_hist=True, plot_pred_ints=True, plot_post_pred_means=True)

TODO better for another counter site in the same city in the same year than for same counter site in the next year
TODO possible reason: Covid, Freiburg ist Uni-Stadt --> more bike riders in 2022 because students are back in town

In [ ]:
print('MAE when predicting mean:', np.median(np.abs(test_data_new_counter_site['rider_count'] - np.mean(test_data_new_counter_site['rider_count']))))
print('Standard deviation of absolute errors when predicting mean:', np.std(np.abs(test_data_new_counter_site['rider_count'] - np.mean(test_data_new_counter_site['rider_count']))))
# TODO 890 vs. 842 --> Heißt das, MAE unseres Modells kaum (numerisch) besser als einfach Mittelwert der Daten vorhersagen? (an neuer counting station)?
# TODO und Streuung wäre auch noch kleiner?
# TODO Vergleiche mit Modell, das extra für diesen Standort gefittet wurde. Das sollte dann schon deutlich besser sein als einfach nur Mittelwert vorhersagen, oder?

##### Test for another city

TODO search for city for which data for whole 2021 is given and a counting station where there is a 1-to-1-mapping between channel name and channel ID.

In [ ]:
# specify counting station and time period
test_city_name = 'Stadt Ludwigsburg'
test_counter_site = 'Solitudeallee'
test_channel_name = 'Solitudeallee Stadteinwärts'
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 12, 31)
#test_channel_ID = 101014511
# TODO use channel_id for identification --> to be able to do so: export also channel_id when building the dataset

# extract data for counting station
test_data_new_city = mdl.get_data_for_location(test_city_name, test_counter_site, test_channel_name, start_date, end_date)

In [ ]:
# check data import
test_data_new_city

In [ ]:
# test model on data of another city
# compare the posterior predictive means with observed (test) data points
linCatMod_ppc_new_city, linCatMod_posterior_predicitive_means_new_city, linCatMod_errors_new_city, linCatMod_median_absolute_error_new_city, linCatMod_scaled_MAE_new_city, linCatMod_quants_per_datapoint_df_new_city, linCatMod_percent_within_50_new_city, linCatMod_percent_within_95_new_city = m_check.post_pred_check(linearCategoricalModel, linearCategoricalModel_idata, given_data = test_data_new_city, change_dat=True, print_output=True, plot_error_hist=True, plot_pred_ints=True, plot_post_pred_means=True)

TODO scaling passt gar nicht.

#### Fit and test model for normalized rider counts

In [ ]:
# TODO use different normalization?
# TODO heißt das "normalize"? "standardize"?
# TODO standardize? Wie heißt das, wenn Ergebnis in [0, 1]?
def normalize_rider_count(dat):
    '''
    Normalize the rider counts to [0,1].

    Parameters
    ----------
    dat : pd.DataFrame
        pandas Dataframe with columns 'temperature', 'is_busday', 'rider_count'
    
    Returns
    -------
    pd.DataFrame
    '''
    dat_normalized_rider_counts = dat.copy()
    dat_normalized_rider_counts['rider_count'] = (dat_normalized_rider_counts['rider_count'] - min(dat_normalized_rider_counts['rider_count'])) / (max(dat_normalized_rider_counts['rider_count']) - min(dat_normalized_rider_counts['rider_count']))

    return dat_normalized_rider_counts

##### Build model for normalized rider counts

In [ ]:
# normalize rider counts
dat_normalized_rider_counts = normalize_rider_count(combined_daily_dat)

# build linear model with categories "business-day vs no-business-day"
prior_config_normalized = m_helpers.create_prior_config_dict(0, 1, 0, 1, 0, 1, 0, 1) # TODO adjust prior
linearCategoricalModelForNormalized, linearCategoricalModelForNormalized_idata = build_linear_model_with_normal_data_distrib(prior_config_normalized, dat=dat_normalized_rider_counts)

In [ ]:
m_check.post_pred_check(linearCategoricalModelForNormalized, linearCategoricalModelForNormalized_idata, given_data = dat_normalized_rider_counts, print_output=True, plot_error_hist=True, plot_pred_ints=True, plot_post_pred_means=True);
# TODO adjust y-axis label

##### Test model

###### Test model on normalized data for new counter site

In [ ]:
# normalize
test_data_new_counter_site_normalized = normalize_rider_count(test_data_new_counter_site)

# test model on new data
linCatModForNormalized_ppc_new_counter_site, linCatModForNormalized_posterior_predicitive_means_new_counter_site, linCatModForNormalized_errors_new_counter_site, linCatModForNormalized_median_absolute_error_new_counter_site, linCatModForNormalized_scaled_MAE_new_counter_site, linCatModForNormalized_quants_per_datapoint_df_new_counter_site, linCatModForNormalized_percent_within_50_new_counter_site, linCatModForNormalized_percent_within_95_new_counter_site = m_check.post_pred_check(linearCategoricalModelForNormalized, linearCategoricalModelForNormalized_idata, given_data = test_data_new_counter_site_normalized, change_dat=True, print_output=True, plot_error_hist=True, plot_pred_ints=True, plot_post_pred_means=True)

Plot prediction intervals and data for original data and normalized data side by side.

In [ ]:
mplot.plot_data_with_pred_intervals(linCatMod_quants_per_datapoint_df_new_counter_site, dat=test_data_new_counter_site)
mplot.plot_data_with_pred_intervals(linCatModForNormalized_quants_per_datapoint_df_new_counter_site, dat=test_data_new_counter_site_normalized, ylabel='Normalized daily number of bike riders')

###### Test model on normalized data for new city

In [ ]:
# normalize
test_data_new_city_normalized = normalize_rider_count(test_data_new_city)

# test model on new data
linCatModForNormalized_ppc_new_city, linCatModForNormalized_posterior_predicitive_means_new_city, linCatModForNormalized_errors_new_city, linCatModForNormalized_median_absolute_error_new_city, linCatModForNormalized_scaled_MAE_new_city, linCatModForNormalized_quants_per_datapoint_df_new_city, linCatModForNormalized_percent_within_50_new_city, linCatModForNormalized_percent_within_95_new_city = m_check.post_pred_check(linearCategoricalModelForNormalized, linearCategoricalModelForNormalized_idata, given_data = test_data_new_city_normalized, change_dat=True, print_output=True, plot_error_hist=True, plot_pred_ints=True, plot_post_pred_means=True)

Plot prediction intervals and data for original data and normalized data side by side.

In [ ]:
mplot.plot_data_with_pred_intervals(linCatMod_quants_per_datapoint_df_new_city, dat=test_data_new_city)
mplot.plot_data_with_pred_intervals(linCatModForNormalized_quants_per_datapoint_df_new_city, dat=test_data_new_city_normalized, ylabel='Normalized daily number of bike riders')

### Poisson Regression: Poisson model with identity link function

TODO  
Reasonable? Might be that lambda-parameter-value of Poisson distribution is <0...  (--> we assume that this is not an issue for our "region of interest" (i.e., from -5°C to 25°C))  
Poisson-Verteilung der Bike-Rider-Counts? Der Residuen?

In [ ]:
def build_linear_model_with_poisson_data_distrib(prior_config, dat=combined_daily_dat):
    '''
    Build a linear model with categories "business-day vs no-business-day",
    using the given parameter values for the prior distributions and a Poisson
    distribution as data distribution.

    Parameters
    ----------
    prior_config : dict
        Dictionary that contains the parameter values of the prior distributions.
    dat : pd.DataFrame
        Dataframe with columns 'temperature', 'is_busday' and 'rider_count'.
        Default: combined_daily_dat

    Returns
    -------
    model : pm.Model
        Model
    idata : arviz.InferenceData
        samples from the posterior
    '''
    # build linear poisson (TODO wie heißt das jetzt?) model with categories "business-day vs no-business-day"
    with pm.Model() as model:

        # data
        dat_temps = pm.Data('dat_temps', dat[['temperature']])
        dat_busday = pm.Data('dat_busday', dat[['is_busday']])
        dat_rider_counts = pm.Data('dat_rider_counts', dat[['rider_count']])
        
        # prior for parameters beta
        # idea: same parametrization as in Frequentist analysis above.
        if (prior_config['truncate_intercept_prior']): # truncate intercept prior: >= 0.
            beta_intercept = pm.TruncatedNormal("beta_intercept", mu=prior_config['mu_intercept_prior'], sigma=prior_config['sigma_intercept_prior'], lower=0, shape=1)
        else:
            beta_intercept = pm.Normal("beta_intercept", mu=prior_config['mu_intercept_prior'], sigma=prior_config['sigma_intercept_prior'], shape=1)
        beta_slope = pm.Normal("beta_slope", mu=prior_config['mu_slope_prior'], sigma=prior_config['sigma_slope_prior'], shape=1)
        beta_intercept_business_days = pm.Normal("beta_intercept_business_days", mu=prior_config['mu_intercept_business_days_prior'], sigma=prior_config['sigma_intercept_business_days_prior'], shape=1)
        beta_slope_interaction = pm.Normal("beta_slope_interaction", mu=prior_config['mu_slope_interaction_prior'], sigma=prior_config['sigma_slope_interaction_prior'], shape=1)
            
        # the linear model function
        expected = beta_intercept + beta_slope * dat_temps + beta_intercept_business_days * dat_busday + beta_slope_interaction * dat_temps * dat_busday
        # TODO hier vielleicht noch sicherstellen, dass es >= 0 ist? Für Poisson...

        # Poisson likelihood
        obs = pm.Poisson("observed",
                        mu=expected,
                        observed=dat_rider_counts)

        # sample from posterior
        idata = pm.sample(2000, chains=4, tune=3000, target_accept=0.9, return_inferencedata=True)

    return model, idata

In [ ]:
# prior for parameter vector beta
prior_config_poisson = m_helpers.create_prior_config_dict(4000, 2000, 300, 300, 0, 2000, -30, 100)
linearPoissonModel, linearPoissonModel_idata = build_linear_model_with_poisson_data_distrib(prior_config_poisson, dat=combined_daily_dat)

In [ ]:
# print / plot the fit results
mplot.show_results_linear_fit(linearPoissonModel_idata)

--> TODO interaction ist hier signifikant. Liegt das an prior?

##### Posterior predictive check

In [ ]:
# posterior predictive check
m_check.post_pred_check(linearPoissonModel, linearPoissonModel_idata, print_output=True, plot_pred_ints=True, plot_post_pred_means=False, quant0=True);

TODO --> coverage ist miserabel. Model passt vermutlich nicht.

TODO prediction viel zu wenig Varianz, oder? Data overdispersed?

### Negative Binomial Regression: Negative binomial model with identity link function

TODO Since data seems overdispersed (for Poisson), try it with negative binomial distribution as data distribution.

In [ ]:
def build_linear_model_with_negative_binomial_data_distrib(prior_config, dat=combined_daily_dat):
    '''
    Build a linear model with categories "business-day vs no-business-day",
    using the given parameter values for the prior distributions and a Negative
    Binomial distribution as data distribution.

    Parameters
    ----------
    prior_config : dict
        Dictionary that contains the parameter values of the prior distributions.
    dat : pd.DataFrame
        Dataframe with columns 'temperature', 'is_busday' and 'rider_count'.
        Default: combined_daily_dat

    Returns
    -------
    model : pm.Model
        Model
    idata : arviz.InferenceData
        samples from the posterior
    '''
    # build linear negative binomial (TODO wie heißt das jetzt?) model with categories "business-day vs no-business-day"
    with pm.Model() as model:

        # data
        dat_temps = pm.Data('dat_temps', dat[['temperature']])
        dat_busday = pm.Data('dat_busday', dat[['is_busday']])
        dat_rider_counts = pm.Data('dat_rider_counts', dat[['rider_count']])
        
        # prior for parameters beta
        # idea: same parametrization as in Frequentist analysis above.
        if (prior_config['truncate_intercept_prior']): # truncate intercept prior: >= 0.
            beta_intercept = pm.TruncatedNormal("beta_intercept", mu=prior_config['mu_intercept_prior'], sigma=prior_config['sigma_intercept_prior'], lower=0, shape=1)
        else:
            beta_intercept = pm.Normal("beta_intercept", mu=prior_config['mu_intercept_prior'], sigma=prior_config['sigma_intercept_prior'], shape=1)
        beta_slope = pm.Normal("beta_slope", mu=prior_config['mu_slope_prior'], sigma=prior_config['sigma_slope_prior'], shape=1)
        beta_intercept_business_days = pm.Normal("beta_intercept_business_days", mu=prior_config['mu_intercept_business_days_prior'], sigma=prior_config['sigma_intercept_business_days_prior'], shape=1)
        beta_slope_interaction = pm.Normal("beta_slope_interaction", mu=prior_config['mu_slope_interaction_prior'], sigma=prior_config['sigma_slope_interaction_prior'], shape=1)
            
        # the linear model function
        expected = beta_intercept + beta_slope * dat_temps + beta_intercept_business_days * dat_busday + beta_slope_interaction * dat_temps * dat_busday

        # Negative binomial likelihood
        alphas = pm.HalfNormal("alpha", 4, shape=2)
        alpha = alphas[0] * (1 - dat_busday) + alphas[1] * dat_busday
        obs = pm.NegativeBinomial("observed",
                        mu=expected, # TODO does this make sense?
                        alpha=alpha,
                        observed=dat_rider_counts)

        # sample from posterior
        idata = pm.sample(2000, chains=4, tune=3000, target_accept=0.9, return_inferencedata=True)

    return model, idata

In [ ]:
# build and fit model
prior_config_neg_binom = m_helpers.create_prior_config_dict(4000, 2000, 300, 300, 0, 2000, -30, 100)
linearNegBinomModel, linearNegBinomModel_idata = build_linear_model_with_negative_binomial_data_distrib(prior_config_neg_binom, dat=combined_daily_dat)

In [ ]:
# print / plot the fit results
mplot.show_results_linear_fit(linearNegBinomModel_idata)

--> TODO Interaction ist hier signifikant. Liegt das an prior? 

##### Posterior predictive check

In [ ]:
# posterior predictive check
m_check.post_pred_check(linearNegBinomModel, linearNegBinomModel_idata, print_output=True, plot_pred_ints=True, plot_post_pred_means=False, quant0=True);

--> Much better than using Poisson data distribution. But still the issue: Negative binomial... variance increses with mean - which does not seem to be the case in the training data.

TODO beschreiben

---
TODO  
Notes:

Possible shortcomings of our analyis method / model:  

- linear model: maybe yields predictions < 0 (--> we assume that this is not an issue for our "region of interest" (i.e., from -5°C to 25°C))
- Normal distribution? --> maybe yields predictions that are not integers.
- Poisson for count data would be good. But: Assumption is violated: Mean $\neq$ Variance

To think about:  
Sind unsere Datenpunkte unabhängig voneinander? Wenn ich Fahrrad schonmal in Keller gebracht habe (für Winter), hole ich es vielleicht auch nicht mehr raus, auch wenn es nochmal wärmer werden sollte...